In [2]:
import torch
import torch.nn as nn

In [18]:
def scaled_dot_product_attention(query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask=None) -> torch.Tensor:
    """
        Each of the input tensor are 3D of shape {batch_size, sequence_length, num_features}
    """
    score = query.bmm(key.transpose(1, 2))   # bmm is batch matrix multiplication. [b, s, f] @ [b, f, s] --> [b, s, s].
    dk = key.shape[-1]
    scaled = score * (dk ** -0.5)

    if mask:
        """
            This condition differentiates between encoder and decoder block. If no mask then every node is allowed to talk to other nodes.
            Masking allows to restrict some nodes from talking to other which is useful in case of decoder where words of the future are not
            allowed to influence current word.
        """  
        scaled = scaled + mask

    attention_weights = torch.softmax(scaled, dim=-1)
    out = attention_weights.bmm(value)
    return out

In [20]:
# Testing the self attention code.
Q = torch.randn((5, 3, 4))
K = torch.randn((5, 3, 4))
V = torch.randn((5, 3, 6))

out = scaled_dot_product_attention(Q, K, V)

In [21]:
print(out.shape)

torch.Size([5, 3, 6])


In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, dim_in: int, dim_q: int, dim_k: int):
        